In [3]:
import os
import pickle
import requests
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, balanced_accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
boay = pd.read_csv('../../1-data-collection/data/boay00.csv')
cho = pd.read_csv('../../1-data-collection/data/cho_shmo.csv')
vinu = pd.read_csv('../../1-data-collection/data/vinu.csv')
oski = pd.read_csv('../../1-data-collection/data/oski.csv')
eden = pd.read_csv('../../1-data-collection/data/eden.csv')


In [5]:
cho.drop(columns = 'Unnamed: 0', inplace = True)
boay.drop(columns = 'Unnamed: 0', inplace = True)
vinu.drop(columns = 'Unnamed: 0', inplace = True)
oski.drop(columns = 'Unnamed: 0', inplace = True)
eden.drop(columns = 'Unnamed: 0', inplace = True)

In [6]:
# res = requests.get('https://ballchasing.com/api/a402545e-5f1b-4446-b58d-9cb91b137aae',
#                   headers={
#                       'Authorization': 'mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5'})

# data = res.json()
# data

# player_data = data['orange']['players'][0]['stats']
# player_data = dict(player_data, **player_data['core'])
# player_data = dict(player_data, **player_data['boost'])
# player_data = dict(player_data, **player_data['movement'])
# player_data = dict(player_data, **player_data['positioning'])
# player_data = dict(player_data, **player_data['demo'])
# del player_data['core']
# del player_data['boost']
# del player_data['movement']
# del player_data['positioning']
# del player_data['demo']
# player_data['player_name'] = 'Vatira'



# df = pd.DataFrame(columns = player_data.keys())
# for j in range(0, min([len(data['orange']['players']), len(data['blue']['players'])])):
            
               
#     if data['orange']['players'][j]['name'].lower() == 'Vatira'.lower():
#         player_data = data['orange']['players'][j]['stats']

#         player_data = dict(player_data, **player_data['core'])
#         player_data = dict(player_data, **player_data['boost'])
#         player_data = dict(player_data, **player_data['movement'])
#         player_data = dict(player_data, **player_data['positioning'])
#         player_data = dict(player_data, **player_data['demo'])
#         del player_data['core']
#         del player_data['boost']
#         del player_data['movement']
#         del player_data['positioning']
#         del player_data['demo']

#         player_data['title'] = data['title']
#         player_data['player_name'] = 'Vatira'
#         df.loc[player_data['title'], :] = player_data

#     elif data['blue']['players'][j]['name'].lower() == 'Vatira'.lower():

#         player_data = data['blue']['players'][j]['stats']

#         player_data = dict(player_data, **player_data['core'])
#         player_data = dict(player_data, **player_data['boost'])
#         player_data = dict(player_data, **player_data['movement'])
#         player_data = dict(player_data, **player_data['positioning'])
#         player_data = dict(player_data, **player_data['demo'])
#         del player_data['core']
#         del player_data['boost']
#         del player_data['movement']
#         del player_data['positioning']
#         del player_data['demo']

#         player_data['title'] = data['title']
#         player_data['player_name'] = 'Vatira'

#         df.loc[player_data['title'], :] = player_data
# df.mvp = df.mvp.astype(int)
# df

In [7]:
with open('../../3-modeling-playstyle/code/et_model.pkl', 'rb') as picklefile:
    et_model = pickle.load(picklefile)

In [8]:
with open('../../3-modeling-playstyle/code/nn_model.pkl', 'rb') as picklefile:
    nn_model = pickle.load(picklefile)

In [9]:
add_nn = load_model('../../3-modeling-playstyle/code/tf_save_model')

In [10]:
to_drop = [
    'shots_against',
    'goals_against',
    'shooting_percentage',
    'bpm',
    'amount_stolen_big',
    'amount_stolen_small',
    'count_collected_big',
    'count_collected_small',
    'count_stolen_small',
    'count_stolen_big',
    'amount_overfill_stolen',
    'time_zero_boost',
    'time_full_boost',
    'time_boost_0_25',
    'time_boost_25_50',
    'time_boost_50_75',
    'time_boost_75_100',
    'avg_speed',
    'total_distance',
    'time_supersonic_speed',
    'time_boost_speed',
    'time_slow_speed',
    'time_ground',
    'time_low_air',
    'time_high_air',
    'time_powerslide',
    'time_defensive_third',
    'time_neutral_third',
    'time_offensive_third',
    'time_defensive_half',
    'time_offensive_half',
    'time_behind_ball',
    'time_infront_ball',
    'time_most_back',
    'time_most_forward',
    'time_closest_to_ball',
    'time_farthest_from_ball',
#     'goals_against_while_last_defender',
    'mvp',
    'taken',
    'goals',
    'assists',
    'avg_distance_to_ball_possession',
    'shots',
    'amount_stolen'
]

In [11]:
boay.drop(columns = to_drop, inplace = True)
cho.drop(columns = to_drop, inplace = True)
vinu.drop(columns = to_drop, inplace = True)
oski.drop(columns = to_drop, inplace = True)
eden.drop(columns = to_drop, inplace = True)


In [12]:
for col in boay.columns[:-1]:
    boay[col] = boay[col].astype(float)

for col in cho.columns[:-1]:
    cho[col] = cho[col].astype(float)
    
for col in vinu.columns[:-1]:
    vinu[col] = vinu[col].astype(float)
    
for col in oski.columns[:-1]:
    oski[col] = oski[col].astype(float)
    
for col in eden.columns[:-1]:
    eden[col] = eden[col].astype(float)

In [13]:
cho.drop(columns = 'goals_against_while_last_defender', inplace = True)
vinu.drop(columns = 'goals_against_while_last_defender', inplace = True)

In [14]:
eden.drop(columns = 'goals_against_while_last_defender', inplace = True)

In [30]:
poly = PolynomialFeatures(interaction_only= True)
poly.fit(oski.dropna().drop(columns = 'player_name'))
boay_poly = pd.DataFrame(poly.transform(boay.drop(columns = 'player_name')), columns=poly.get_feature_names_out())
cho_poly = pd.DataFrame(poly.transform(cho.drop(columns = 'player_name')), columns=poly.get_feature_names_out())
vinu_poly = pd.DataFrame(poly.transform(vinu.drop(columns = 'player_name')), columns=poly.get_feature_names_out())
oski_poly = pd.DataFrame(poly.transform(oski.dropna().drop(columns = 'player_name')), columns=poly.get_feature_names_out())
eden_poly = pd.DataFrame(poly.transform(eden.dropna().drop(columns = 'player_name')), columns=poly.get_feature_names_out())

In [31]:
cho_preds = pd.DataFrame(et_model.predict_proba(cho_poly), columns = ['Monkeymoon','Oski','Vati'])

In [32]:
cho_preds

,Monkeymoon,Oski,Vati
0,0.509218,0.085376,0.405406
1,0.501972,0.085376,0.412653
2,0.686609,0.088253,0.225138
3,0.512258,0.124819,0.362923
4,0.630175,0.186994,0.182831
5,0.609659,0.069012,0.321329
6,0.570652,0.205981,0.223366
7,0.602001,0.219927,0.178072
8,0.587306,0.266613,0.146081
9,0.765655,0.052564,0.181781


In [35]:
pd.DataFrame(et_model.predict(cho_poly)).value_counts()

M0nkey M00n    18
Vati            1
dtype: int64

In [36]:
cho_preds.mean()

Monkeymoon    0.584503
Oski          0.137637
Vati          0.277860
dtype: float64

In [37]:
boay_preds = pd.DataFrame(et_model.predict_proba(boay_poly), columns = ['Monkeymoon','Oski','Vati'])

In [38]:
boay_preds.mean()

Monkeymoon    0.463577
Oski          0.181957
Vati          0.354466
dtype: float64

In [39]:
pd.DataFrame(et_model.predict(boay_poly)).value_counts()

M0nkey M00n    11
Vati            7
Oski            1
dtype: int64

In [40]:
vinu_preds = pd.DataFrame(et_model.predict_proba(vinu_poly), columns = ['Monkeymoon','Oski','Vati'])

In [41]:
vinu_preds.mean()

Monkeymoon    0.609793
Oski          0.120981
Vati          0.269226
dtype: float64

In [42]:
pd.DataFrame(et_model.predict(vinu_poly)).value_counts()

M0nkey M00n    29
Vati            5
dtype: int64

In [43]:
oski.dropna(inplace = True)

In [44]:
oski_preds = pd.DataFrame(et_model.predict_proba(oski_poly), columns = ['Monkeymoon','Oski','Vati'])

In [45]:
oski_preds.mean()

Monkeymoon    0.020366
Oski          0.910927
Vati          0.068707
dtype: float64

In [47]:
oski_poly

,1,saves,score,bcpm,avg_amount,amount_collected,amount_collected_big,amount_collected_small,amount_overfill,amount_used_while_supersonic,...,percent_most_back percent_most_forward,percent_most_back percent_closest_to_ball,percent_most_back percent_farthest_from_ball,percent_most_back inflicted,percent_most_forward percent_closest_to_ball,percent_most_forward percent_farthest_from_ball,percent_most_forward inflicted,percent_closest_to_ball percent_farthest_from_ball,percent_closest_to_ball inflicted,percent_farthest_from_ball inflicted
0,1.0,0.0,378.0,420.76846,48.78,2338.0,1535.0,803.0,296.0,389.0,...,1017.532324,1046.390898,1241.987946,0.000000,838.522033,995.263108,0.000000,1023.490098,0.000000,0.000000
1,1.0,1.0,337.0,443.67334,49.62,2749.0,1535.0,1214.0,280.0,457.0,...,995.167363,1063.863565,1176.191258,34.511513,888.902364,982.756835,28.835808,1050.596340,30.826338,34.081127
2,1.0,2.0,396.0,483.60580,42.02,2677.0,1905.0,772.0,200.0,335.0,...,1058.065785,958.544799,874.737590,57.989340,1206.392824,1100.915839,72.983468,997.364404,66.118690,60.337820
3,1.0,2.0,408.0,427.79123,43.45,2624.0,1868.0,756.0,241.0,315.0,...,886.206533,962.022653,833.977578,31.002907,886.983234,768.925895,28.584627,834.708504,31.030079,26.899980
4,1.0,2.0,212.0,489.84360,42.65,2725.0,1900.0,825.0,215.0,445.0,...,1045.246880,932.329735,793.472105,25.465876,1502.696155,1278.890330,41.045000,1140.732878,36.610943,31.158249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,1.0,2.0,320.0,538.30457,51.41,2902.0,2399.0,503.0,355.0,753.0,...,1061.911993,989.486708,1068.763036,63.315402,1048.430736,1132.429579,67.087120,1055.194803,62.511596,67.519940
319,1.0,2.0,339.0,510.09616,46.59,2943.0,2031.0,912.0,391.0,624.0,...,1139.021171,1028.167585,1125.164501,0.000000,1145.194791,1253.232007,0.000000,1131.263016,0.000000,0.000000
320,1.0,0.0,298.0,480.92430,44.31,2893.0,2110.0,783.0,308.0,420.0,...,1146.728172,1223.107005,1147.774411,37.763557,983.510831,922.935246,30.366000,984.408156,32.388554,30.393705
321,1.0,1.0,181.0,469.27370,45.52,1512.0,1247.0,265.0,262.0,118.0,...,957.642078,1117.729833,676.406805,0.000000,1376.943670,833.272980,0.000000,972.570118,0.000000,0.000000


In [48]:
pd.DataFrame(et_model.predict(oski_poly)).value_counts()

Oski           319
Vati             3
M0nkey M00n      1
dtype: int64

In [49]:
eden.dropna(inplace = True)

In [50]:
eden_preds = pd.DataFrame(et_model.predict_proba(eden_poly), columns = ['Monkeymoon','Oski','Vati'])

In [51]:
eden_preds

,Monkeymoon,Oski,Vati
0,0.401908,0.290339,0.307753
1,0.425441,0.301795,0.272764
2,0.657005,0.097826,0.245169
3,0.597826,0.089007,0.313167
4,0.201955,0.161965,0.636080
5,0.211836,0.157024,0.631139
6,0.464923,0.064366,0.470711
7,0.445031,0.066580,0.488389
8,0.339948,0.377754,0.282298
9,0.652507,0.049879,0.297614


In [52]:
eden_preds.mean()

Monkeymoon    0.345436
Oski          0.202005
Vati          0.452559
dtype: float64

In [53]:
pd.DataFrame(et_model.predict(eden_poly)).value_counts()

Vati           28
M0nkey M00n    15
Oski            6
dtype: int64

In [ ]:
nn_predictions = pd.DataFrame(add_nn.predict(boay.drop(columns = 'player_name')), columns = ['MonkeyMoon', 'Oski', 'Vatira'])
nn_predictions

In [ ]:
boay.to_csv('../../3-modeling-playstyle/data/boay_.csv')